In [ ]:
import os
import math
import time 
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pandas import read_csv
from cv2 import imread

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
label_classes = ['森森', '牛排', '巧巧', '小白', '花捲', '金毛', '巧虎', '超人', '吊吊', '黑胖', '乖狗', '橘子']

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    # horizontal_flip=True,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # rotation_range=10,
    # # shear_range=0.1,
    # zoom_range=0.3,
    # validation_split=0.1,
    # fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    directory="../data/",
    # subset='training',
    target_size=(224,224),
    color_mode="rgb",
    class_mode='sparse',
    classes = label_classes,
    batch_size=32,
    # shuffle=True,
)

In [ ]:
# for my_batch in train_generator:
#     images = my_batch[0]
#     labels = my_batch[1]
#     for i in range(len(labels)):
#         plt.imshow(images[i])
#         plt.colorbar()
#         plt.show()
#         print(labels[i])
#     break

In [ ]:
base_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(224,224,3),
    include_top = False,
    weights = 'imagenet',
    classes = 3,
    classifier_activation = "softmax"
)

base_model.trainable = True

inputs = tf.keras.Input(shape=(224,224,3))

x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(12)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
# model.compile(
#     optimizer=tf.keras.optimizers.Adam,
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
# )

# my_callback = [tf.keras.callbacks.ModelCheckpoint(
#     filepath='model_save/model_{epoch:03d}'
# )]

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=10,
)

acc = history.history['accuracy']
epochs = range(1, len(acc) + 1)
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc', linewidth=0.5)
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.grid()
# 儲存acc學習曲線
# plt.savefig('./acc.png')
plt.show()

# 畫出loss學習曲線
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss', linewidth=0.5)
plt.title('Training and validation loss')
plt.legend(loc='upper right')
plt.grid()
# 儲存loss學習曲線
# plt.savefig('loss.png')
plt.show()